Below code takes <tt>CurrencyRates.xml</tt> that comes with <i>"Декларация"</i> russian tax software and makes a list of available currencies from it.<br>
Then it takes a list of currencies from Russian Central bank site.<br>
Both list are merged in order to create <tt>currencies</tt> dictionary for <tt>DLSG</tt> class.

In [38]:
import requests
import pandas as pd
from lxml import etree

In [39]:
rows = []
for currency in etree.parse('./CurrencyRates.xml').findall('/Currency'):
    rows.append({"code": currency.attrib['Code'], "name": currency.attrib['Name'], "multiplier": currency.xpath('TendersDateRate')[0].attrib['Quantity']})
TAX_codes = pd.DataFrame(rows, columns=["code", "name", "multiplier"])

In [40]:
rows = []
cbr_list = etree.fromstring(bytes(requests.get("http://www.cbr.ru/scripts/XML_valFull.asp").text, encoding='cp1251'))
for node in cbr_list:
    if node.find("ISO_Num_Code").text is None:
        continue
    code = f"{int(node.find('ISO_Num_Code').text):03d}"
    iso = node.find("ISO_Char_Code").text
    rows.append({"iso": iso, "code": code})
CBR_codes = pd.DataFrame(rows, columns=["iso", "code"])

In [41]:
currencies = pd.merge(left=CBR_codes, right=TAX_codes, on="code")
print("currencies = {")
for i, row in currencies.iterrows():
    line_end = '' if i == (currencies.shape[0] - 1) else ','
    print(f"    '{row['iso']}': {{'code': '{row['code']}', 'name': '{row['name']}', 'multiplier': {row['multiplier']}}}" + line_end)
print("}")

currencies = {
    'AUD': {'code': '036', 'name': 'Австралийский доллар', 'multiplier': 100},
    'AZN': {'code': '944', 'name': 'Азербайджанский манат', 'multiplier': 100},
    'GBP': {'code': '826', 'name': 'Фунт стерлингов', 'multiplier': 100},
    'AMD': {'code': '051', 'name': 'Армянский драм', 'multiplier': 10000},
    'BYN': {'code': '933', 'name': 'Белорусский рубль', 'multiplier': 100},
    'BGN': {'code': '975', 'name': 'Болгарский лев', 'multiplier': 100},
    'BRL': {'code': '986', 'name': 'Бразильский реал', 'multiplier': 100},
    'HUF': {'code': '348', 'name': 'Форинт', 'multiplier': 10000},
    'HKD': {'code': '344', 'name': 'Гонконгский доллар', 'multiplier': 1000},
    'DKK': {'code': '208', 'name': 'Датская крона', 'multiplier': 100},
    'USD': {'code': '840', 'name': 'Доллар США', 'multiplier': 100},
    'EUR': {'code': '978', 'name': 'Евро', 'multiplier': 100},
    'INR': {'code': '356', 'name': 'Индийская рупия', 'multiplier': 10000},
    'KZT': {'code': '398', '